In [24]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time


In [25]:
driver = Driver(uc=True)   # undetected-chromedriver

In [26]:
BASE_URL = 'https://www.noon.com/egypt-ar/search/?page={}&q=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA%20%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87%20%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85%20%D8%AF%D8%B4'

In [27]:
all_product_links = []
driver.implicitly_wait(10)

In [28]:
for i in range(1,19):
    driver.get(BASE_URL.format(i))
    time.sleep(3) 
    PRODUCT_NAME_LOCATOR = (By.XPATH, '//h2[@data-qa="plp-product-box-name"]')
    product_name_elements = driver.find_elements(*PRODUCT_NAME_LOCATOR)
    
    if not product_name_elements:
        print("No product elements found on the page.")
    
    for index, name_element in enumerate(product_name_elements):
        link_element = name_element.find_element(By.XPATH, './ancestor::a[1]')
        product_link = link_element.get_attribute('href')
        all_product_links.append(product_link)


In [29]:
all_product_links

['https://www.noon.com/egypt-ar/2-in-1-floor-brush-long-handled-v-shaped-floor-brush-bathroom-shower-crevice-cleaning-brush-magic-broom-brush-120-rotation-detachable-bathroom-brush-head-tile/Z415B2A93E0CA87DD508EZ/p/?o=z415b2a93e0ca87dd508ez-1',
 'https://www.noon.com/egypt-ar/bathroom-shower-head-with-five-water-pumping-modes-different-forms/Z6E5E06C804A132949392Z/p/?o=z6e5e06c804a132949392z-1',
 'https://www.noon.com/egypt-ar/turbo-shower-head-high-pressure-handheld-showerhead-adjustable-water-saving-high-pressure-turbo-showerhead-with-5-modes-and-easy-on-off-button-handheld-shower-head-for-bathroom/ZC00D0704912DC6A5CCFBZ/p/?o=dba937f449450a5e',
 'https://www.noon.com/egypt-ar/30-piece-shower-nozzle-cleaning-brush-anti-clogging-nylon-cleaning-brush-gap-hole-cleaning-brush-for-bathroom-shower-cleaning-tool/Z2099D60957232BC0A945Z/p/?o=z2099d60957232bc0a945z-1',
 'https://www.noon.com/egypt-ar/ultra-thin-8-inchessquare-stainless-steel-bathroom-shower-head/Z1225D88949A1808FC20CZ/p/?o=z12

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def extract_noon_reviews(driver, url, timeout=15):
    """
    Extract all reviews from a Noon product page.
    
    Args:
        driver: Selenium WebDriver
        url: Product URL
        timeout: Max seconds to wait for reviews to load

    Returns:
        List of dictionaries with review info
    """
    driver.get(url)
    all_reviews = []
    wait = WebDriverWait(driver, timeout)
    product_name = driver.find_element(By.CSS_SELECTOR, "[class*='ProductTitle']").text

    # ---- Gradual scroll until reviews section is visible ----
    def scroll_to_reviews():
        for _ in range(5):
            try:
                review_section = driver.find_element(By.XPATH, "//div[contains(@class,'ProductReviewsFilters-module') and contains(., 'المراجعات')]")
                if review_section.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
                    return True
            except:
                pass
            driver.execute_script("window.scrollBy(0, 550);")
            time.sleep(1.1)
        return False

    if not scroll_to_reviews():
        print("⚠️ Reviews section not found on page")
        return []

    time.sleep(1)  # small wait for rendering

    # ---- Click "View All Reviews" if exists ----
    try:
        view_all = driver.find_element(By.CSS_SELECTOR, ".ViewAllReviewsBtn-module-scss-module__viewMore")
        driver.execute_script("arguments[0].scrollIntoView(true);", view_all)
        time.sleep(0.5)
        view_all.click()
        time.sleep(2)  # wait for all reviews page to load
    except NoSuchElementException:
        pass  # no "View All Reviews" button

    # ---- Wait until at least one review is visible or timeout ----
    try:
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem")
            )
        )
    except TimeoutException:
        print("⚠️ No reviews found for this product")
        return []

    # ---- Extract reviews ----
    review_containers = driver.find_elements(
        By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem"
    )

    for r in review_containers:
        try:
            username = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__userName").text
        except:
            username = "anonymous"

        try:
            date = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__ratedDate").text
        except:
            date = ""

        try:
            title = r.find_element(By.CSS_SELECTOR, ".TranslatedReviewTitle-module-scss-module__QbaJfW__reviewTitle").text
        except:
            title = ""

        try:
            body = r.find_element(By.CSS_SELECTOR, ".ReviewDescription-module-scss-module__AkZflW__reviewDesc").text
        except:
            body = ""

        try:
            rating_imgs = r.find_elements(By.CSS_SELECTOR, ".RatingStars-module-scss-module__fF13gq__container img")
            rating = sum(1 for img in rating_imgs if "rating" in img.get_attribute("color") and "grey" not in img.get_attribute("color"))
        except:
            rating = None

        all_reviews.append({
            "product": product_name,
            "username": username,
            "date": date,
            "title": title,
            "text": body,
            "rating": rating
        })

    print(f"✅ Found {len(all_reviews)} review(s)")
    return all_reviews


In [31]:
# url = "https://www.noon.com/egypt-ar/loq-gaming-laptop-with-15-6-inch-full-hd-1920x1080-ips-display-core-i5-12450hx-processor-16gb-ddr5-ram-512gb-ssd-4gb-nvidia-geforce-rtx-2050-graphics-dos-without-windows-english-arabic-grey/N70148823V/p/?utm_source=C1000088L&utm_medium=cpc&utm_campaign=C1000151355N_eg_en_web_searchxxexactandphrasexxbrandpurexx08082022_noon_web_c1000088l_acquisition_sembranded&o=caaa680fb6a8d82a&shareId=5ea23957-22eb-40d3-8a33-91ee8940ab29"
# data = extract_noon_reviews(driver,url)

In [32]:
# data

In [33]:
all_reviews = []
for url in all_product_links:
    reviews = extract_noon_reviews(driver,url)
    all_reviews.extend(reviews)  # Combine all reviews in one list

# Convert to DataFrame



⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
✅ Found 3 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
✅ Found 1 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on pag

ProtocolError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [34]:
df_all_reviews = pd.DataFrame(all_reviews)
df_all_reviews.head()

,product,username,date,title,text,rating
0,رأس دش معدني للحمام بتصميم يوفر المياه شفاف 30سم,ام يزن ا.,"يناير 14, 2023",حلو,جميل,5
1,رأس دش معدني للحمام بتصميم يوفر المياه شفاف 30سم,MOHAMED H.,"مارس 19, 2023",عديم الفائده,المي ما تطلع الا خفيييفه والمنتج لا انصح به,1
2,رأس دش معدني للحمام بتصميم يوفر المياه شفاف 30سم,محمد ا.,"مايو 24, 2022",,جداََ ممتاز خدمني في الملحق دفع قوي,5
3,رأس دش حمام بالخرطوم والعليقة,Ahmed M.,"مايو 13, 2025",الجودة,الجودة ليست كما توقعت. يوجد تسريب فى السماعه,4
4,رأس دش حمام بالخرطوم والعليقة,Ahmed M.,"مايو 13, 2025",الجودة,الجودة ليست كما توقعت. يوجد تسريب فى السماعه,4


In [35]:
df_all_reviews.duplicated().sum()

np.int64(1)

In [36]:
df_all_reviews.drop_duplicates(inplace=True)

In [37]:
df_all_reviews.shape

(41, 6)

In [38]:
df_all_reviews.to_csv("all_noon_review_shower.csv", index=False)